# CAS REST - This is an R kernel 
#### Ensure CAS services are running before executing

In [150]:
library(httr)
library(jsonlite)

hostname <- 'sasserver.demo.sas.com:8777'
server <- 'cas-shared-default'              # CAS server name
uri.token <- 'SASLogon/oath/token'
uri.casManagement <- 'casManagement/servers'
uri.casProxy <- 'casProxy/servers'

In [151]:
# Get basic environment info
GET(paste(hostname, 'cas', sep='/'), authenticate('viyauser','Orion123'))

Response [http://sasserver.demo.sas.com:8777/cas]
  Date: 2016-10-27 21:20
  Status: 200
  Content-Type: application/json
  Size: 619 B
{ "type": "Controller",
 "state": "Running",
 "client": 5570,
 "control": 5580,
 "process": {
  "id": "5014",
  "owner": "cas",
  "uptime": 401.273,
  "userCPU": 0.23,
  "sysCPU": 0.06,
...

In [3]:
r <- GET(paste(hostname, 'grid', sep='/'), authenticate('sasdemo','Orion123'))

lapply(content(r), function(x) { paste(x$name, x$type, sep=' - ')})

[[1]]
[1] "racesx11110.demo.sas.com - Controller"

In [155]:
# Create a session and store the id
sess <- content(POST(paste(hostname, 'cas', 'sessions', sep='/'), authenticate('viyauser','Orion123')))$session
print(sess)

[1] "fd3fa77d-4aa2-3e4d-82e6-d135675401dd"


In [156]:
# Specify action parameters
params <- '{"casout": {"caslib": "casuser", "name": "auto_policy"}, "importOptions": {"fileType": "CSV"} }'

# Store the start time
start <- proc.time()

r <- PUT(paste(hostname, 'cas', 'sessions', sess, 'actions', 'table.upload', sep='/'),
         body=upload_file('../JupyterDemos_JW/data/auto_policy.csv'),
         authenticate('viyauser','Orion123'),
         add_headers('JSON-Parameters'=params, 'Content-Type'='binary/octet-stream')
         )

# Determine execution time
print(proc.time() - start)
print(content(r)$metrics$elapsedTime)































































































































































































































































































































































   user  system elapsed 
  0.679   0.000   1.183 
[1] 1.155719


In [160]:
# THIS IS A TEST BLOCK - DELETE WHEN ALL DONE Specify action parameters
params <- '{"casout": {"caslib": "casuser", "name": "cloud-pricing"}, "importOptions": {"fileType": "CSV"} }'

# Store the start time
start <- proc.time()

r <- PUT(paste(hostname, 'cas', 'sessions', sess, 'actions', 'table.upload', sep='/'),
         body=upload_file('../JupyterDemos_JW/data/cloud-pricing.csv'),
         authenticate('viyauser','Orion123'),
         add_headers('JSON-Parameters'=params, 'Content-Type'='binary/octet-stream')
         )

r

Response [http://sasserver.demo.sas.com:8777/cas/sessions/fd3fa77d-4aa2-3e4d-82e6-d135675401dd/actions/table.upload]
  Date: 2016-10-27 21:23
  Status: 200
  Content-Type: application/json
  Size: 1.51 kB
{
  "status": 0,
  "log": "NOTE: Cloud Analytic Services made the uploaded file available as t...
  "results":   {
        "caslib": "CASUSER(viyauser)",
        "tableName": "CLOUD-PRICING"
      },
  "logEntries": [
    { "message": "NOTE: Cloud Analytic Services made the uploaded file availa...
    { "message": "NOTE: The table CLOUD-PRICING has been created in caslib CA...
...

In [159]:
# Helper function for calling CAS actions
callAction <- function(session, action, params, debug=FALSE) {
#    start <- proc.time()

    r <- POST(paste(hostname, 'cas', 'sessions', session, 'actions', action, sep='/'), 
         body=params,
         authenticate('viyauser','Orion123'),
         content_type('application/json'),
         accept_json(),
         encode='json',
         verbose())

    if (debug == TRUE) {
        cat(jsonlite::prettify(rawToChar(r$request$options$postfields)))
    }
#    print(proc.time() - start)
    return(r)
}

In [117]:
x <- content(callAction(sess, 'table.tableInfo', list(caslib='caslib')))
str(x)



List of 6
 $ status          : int 0
 $ log             : chr "ERROR: The caslib 'caslib' does not exist in this session.\nERROR: The action stopped due to errors.\n"
 $ logEntries      :List of 2
  ..$ :List of 2
  .. ..$ message: chr "ERROR: The caslib 'caslib' does not exist in this session."
  .. ..$ level  : chr "error"
  ..$ :List of 2
  .. ..$ message: chr "ERROR: The action stopped due to errors."
  .. ..$ level  : chr "error"
 $ disposition     :List of 5
  ..$ severity       : chr "Error"
  ..$ reason         : chr "OK"
  ..$ statusCode     : int 2710120
  ..$ formattedStatus: chr "The caslib 'caslib' does not exist in this session."
  ..$ debugInfo      : chr "0x887ff878:TKCASTAB_CASLIB_NOTFOUND"
 $ changedResources: list()
 $ metrics         :List of 22
  ..$ elapsedTime               : num 0.000459
  ..$ cpuUserTime               : num 0.001
  ..$ cpuSystemTime             : int 0
  ..$ systemTotalMemory         : num 5.06e+10
  ..$ systemNodes               : int 1
  ..$

In [166]:
# Get table info

getTableInfo <- function(session.p,caslib.p){
    x <- content(callAction(session.p, 'table.tableInfo', list(caslib=caslib.p)))
    keepers <- which(names(unlist(x$results$TableInfo$schema))=='name') 
    res <- data.frame(t(apply(t(r$results$TableInfo$rows),2,FUN=unlist)))
    colnames(res) <- c(t(unlist(x$results$TableInfo$schema)[keepers]))
    return(res)
}

getTableInfo(sess, 'CASUSER')

Name,Rows,Columns,Encoding,CreateTimeFormatted,ModTimeFormatted,JavaCharSet,CreateTime,ModTime,Global,Repeated,View,SourceName,SourceCaslib,Compressed,Creator,Modifier
AUTO_POLICY,83149,21,utf-8,27Oct2016:17:22:28,27Oct2016:17:22:28,UTF8,1793208148.19886,1793208148.19886,0,0,0,,,0,viyauser,
CLOUD-PRICING,64,8,utf-8,27Oct2016:17:23:20,27Oct2016:17:23:20,UTF8,1793208200.38724,1793208200.38724,0,0,0,,,0,viyauser,
BANK-ADDITIONAL-FULL,41188,1,utf-8,27Oct2016:17:23:39,27Oct2016:17:23:39,UTF8,1793208219.86241,1793208219.86241,0,0,0,,,0,viyauser,


In [132]:
uploadCAScsv <- function(session.p, caslib.p, filename, filepath,usr,pwd){
    
    # Specify action parameters
params <- paste('{"casout": {"',caslib.p,'": "casuser", "name":"', filename,'"}, "importOptions": {"fileType": "CSV"} }',sep='')
print(params)

r <- PUT(paste(hostname, 'cas', 'sessions', session.p, 'actions', 'table.upload', sep='/'),
         body=upload_file(paste(filepath,filename,'.csv',sep='')),
         authenticate(usr,pwd),
         add_headers('JSON-Parameters'=params, 'Content-Type'='binary/octet-stream')
         )
return(r)
}



In [162]:
uploadCAScsv(sess,'caslib','bank-additional-full','../JupyterDemos_JW/data/','viyauser','Orion123')

[1] "{\"casout\": {\"caslib\": \"casuser\", \"name\":\"bank-additional-full\"}, \"importOptions\": {\"fileType\": \"CSV\"} }"




































































































































































































































































































































































Response [http://sasserver.demo.sas.com:8777/cas/sessions/fd3fa77d-4aa2-3e4d-82e6-d135675401dd/actions/table.upload]
  Date: 2016-10-27 21:23
  Status: 200
  Content-Type: application/json
  Size: 1.55 kB
{
  "status": 0,
  "log": "NOTE: Cloud Analytic Services made the uploaded file available as t...
  "results":   {
        "caslib": "CASUSER(viyauser)",
        "tableName": "BANK-ADDITIONAL-FULL"
      },
  "logEntries": [
    { "message": "NOTE: Cloud Analytic Services made the uploaded file availa...
    { "message": "NOTE: The table BANK-ADDITIONAL-FULL has been created in ca...
...

In [10]:
# Define a function to extract data from the response and reform as a data frame
createDataFrame <- function(r) {
    # Extract columns names
    header <- lapply(content(r)$results$Fetch$schema, function(x) { x$name })
    #nrows <- length(content(r)$results$Fetch$rows)

    # Combine lists into columns of a matrix
    x <- mapply(c, content(r)$results$Fetch$rows)

    # Transpose and convert to data frame
    x <- as.data.frame(t(x))
    names(x) <- header
    return(x)
}

In [164]:
# Fetch records from table
# NOTE: Specifying caslib causes action to fail?
r <- content(callAction(sess, 'table.tableInfo', list(caslib='CASUSER')))
t(apply(t(r$results$TableInfo$rows),2,FUN=unlist))
#t(t(r$results$TableInfo$rows))

AUTO_POLICY,83149,21,utf-8,27Oct2016:17:22:28,27Oct2016:17:22:28,UTF8,1793208148.19886,1793208148.19886,0,0,0,,,0,viyauser,
CLOUD-PRICING,64,8,utf-8,27Oct2016:17:23:20,27Oct2016:17:23:20,UTF8,1793208200.38724,1793208200.38724,0,0,0,,,0,viyauser,
BANK-ADDITIONAL-FULL,41188,1,utf-8,27Oct2016:17:23:39,27Oct2016:17:23:39,UTF8,1793208219.86241,1793208219.86241,0,0,0,,,0,viyauser,


In [135]:
apply(t(t(r$results$TableInfo$rows)),1,unlist(x),',')

ERROR: Error in match.fun(FUN): 'unlist(x)' is not a function, character or symbol


In [15]:
r <- callAction(sess, 'table.fetch', list(table='AUTO_POLICY', from='1', to='5'))
createDataFrame(r)

_Index_,age,age_ind,deductible_collision_amt,deductible_comprehensive_amt,deductible_comprehensive_cd,driver_training_dt,driver_training_ind,gender_cd,original_inception_dt,⋯,policy_effective_qtr,policy_effective_yr,policy_no,vehicle_no,primary_vehicle_cd,renew_cd,std_occupation_cd,youthful_operator_ind,policy_effective_dt,policy_expiration_dt
1,54,50 - 64,1000,1000,G,NULL,0,F,1987,⋯,2,2005,A05363,2,2,,Police,38,NULL,NULL
2,34,30 - 49,500,500,G,NULL,0,F,1990,⋯,2,2005,A05473,3,3,,Police,18,NULL,NULL
3,49,30 - 49,500,0,F,NULL,0,M,1989,⋯,3,2005,A04279,2,2,,Police,33,NULL,NULL
4,49,30 - 49,1000,500,G,NULL,0,M,1987,⋯,2,2005,A05365,2,2,,Police,33,NULL,NULL
5,46,30 - 49,500,250,G,NULL,0,F,1996,⋯,2,2005,A04610,3,3,,Police,30,NULL,NULL
